In [104]:

import earthkit.data
import earthkit.plots
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
polytope_address="polytope.lumi.apps.dte.destination-earth.eu"    
from scipy.spatial import cKDTree



# Define the request, note that we only extract levtop -> 90 levels to reduce the data amount
levtop = 85
request= {
    "class": "d1",
    "dataset": "on-demand-extremes-dt",
    "expver": "aab0",
    "stream": "oper",
    "date": "2023-08-20",
    "time": "0000",
    "type": "fc",
    "georef": "u09tvk",
    "levtype": "ml",
    "step": [x for x in range(1,24)],
    "levelist": f"{levtop}/to/90",
    "expect" : 0,
    # z,t,u,v,q,w
    "param": ["129", "130", "131", "132", "133", "260238"],
}



In [96]:

data = earthkit.data.from_source("polytope", "destination-earth", request, address=polytope_address, stream=False)

atm = data.sel({"typeOfLevel": "hybrid"}).to_xarray()

atm

2025-11-06 13:49:59 - INFO - Key read from /home/jovyan/.polytopeapirc
2025-11-06 13:49:59 - INFO - Sending request...
{'request': 'class: d1\n'
            'dataset: on-demand-extremes-dt\n'
            "date: '2023-08-20'\n"
            'expect: 0\n'
            'expver: aab0\n'
            'georef: u09tvk\n'
            'levelist: 85/to/90\n'
            'levtype: ml\n'
            'param:\n'
            "- '129'\n"
            "- '130'\n"
            "- '131'\n"
            "- '132'\n"
            "- '133'\n"
            "- '260238'\n"
            'step:\n'
            '- 1\n'
            '- 2\n'
            '- 3\n'
            '- 4\n'
            '- 5\n'
            '- 6\n'
            '- 7\n'
            '- 8\n'
            '- 9\n'
            '- 10\n'
            '- 11\n'
            '- 12\n'
            '- 13\n'
            '- 14\n'
            '- 15\n'
            '- 16\n'
            '- 17\n'
            '- 18\n'
            '- 19\n'
            '- 20\n'
            '- 21\n'


<xarray.Dataset> Size: 861MB
Dimensions:    (step: 3, level: 6, y: 989, x: 989)
Coordinates:
  * step       (step) timedelta64[ns] 24B 01:00:00 02:00:00 03:00:00
  * level      (level) int64 48B 85 86 87 88 89 90
    latitude   (y, x) float64 8MB ...
    longitude  (y, x) float64 8MB ...
Dimensions without coordinates: y, x
Data variables:
    q          (step, level, y, x) float64 141MB ...
    t          (step, level, y, x) float64 141MB ...
    u          (step, level, y, x) float64 141MB ...
    v          (step, level, y, x) float64 141MB ...
    wz         (step, level, y, x) float64 141MB ...
    z          (step, level, y, x) float64 141MB ...
Attributes:
    class:        d1
    stream:       oper
    levtype:      ml
    type:         fc
    expver:       aab0
    date:         20230820
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

In [4]:
atm

<xarray.Dataset> Size: 1GB
Dimensions:    (level: 21, y: 989, x: 989)
Coordinates:
  * level      (level) int64 168B 70 71 72 73 74 75 76 ... 84 85 86 87 88 89 90
    latitude   (y, x) float64 8MB ...
    longitude  (y, x) float64 8MB ...
Dimensions without coordinates: y, x
Data variables:
    q          (level, y, x) float64 164MB 0.009997 0.009956 ... 0.01256 0.01258
    t          (level, y, x) float64 164MB 292.2 292.2 292.1 ... 291.2 291.2
    u          (level, y, x) float64 164MB -1.182 -1.254 ... -0.2198 -0.2633
    v          (level, y, x) float64 164MB 0.3984 0.3296 ... 0.9677 0.9362
    wz         (level, y, x) float64 164MB 0.0114 0.006 ... 0.01644 0.01528
    z          (level, y, x) float64 164MB 9.473e+03 9.485e+03 ... 424.0 434.3
Attributes:
    class:        d1
    stream:       oper
    levtype:      ml
    type:         fc
    expver:       aab0
    date:         20230820
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

In [71]:
LOCATION = ((51.213642, 4.415))
target_lat = LOCATION[0] #51.1954 # Antwerp
target_lon =  LOCATION[1]#4.3615  

lat2d= atm['latitude'].values
lon2d= atm['longitude'].values

# Build KDTree once (re-use it for many lookups)
pts = np.column_stack([lat2d.ravel(), lon2d.ravel()])
tree = cKDTree(pts)

# Query nearest grid cell
_, flat_idx = tree.query([target_lat, target_lon])
iy, ix = np.unravel_index(flat_idx, lat2d.shape)

iy
ix


np.int64(782)

In [75]:
atm

<xarray.Dataset> Size: 861MB
Dimensions:    (step: 3, level: 6, y: 989, x: 989)
Coordinates:
  * step       (step) timedelta64[ns] 24B 01:00:00 02:00:00 03:00:00
  * level      (level) int64 48B 85 86 87 88 89 90
    latitude   (y, x) float64 8MB 46.58 46.58 46.58 46.58 ... 51.02 51.02 51.02
    longitude  (y, x) float64 8MB 359.1 359.1 359.1 359.1 ... 5.865 5.873 5.88
Dimensions without coordinates: y, x
Data variables:
    q          (step, level, y, x) float64 141MB ...
    t          (step, level, y, x) float64 141MB ...
    u          (step, level, y, x) float64 141MB ...
    v          (step, level, y, x) float64 141MB ...
    wz         (step, level, y, x) float64 141MB ...
    z          (step, level, y, x) float64 141MB ...
Attributes:
    class:        d1
    stream:       oper
    levtype:      ml
    type:         fc
    expver:       aab0
    date:         20230820
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

In [92]:

atm

<xarray.Dataset> Size: 861MB
Dimensions:    (step: 3, level: 6, y: 989, x: 989)
Coordinates:
  * step       (step) timedelta64[ns] 24B 01:00:00 02:00:00 03:00:00
  * level      (level) int64 48B 85 86 87 88 89 90
    latitude   (y, x) float64 8MB 46.58 46.58 46.58 46.58 ... 51.02 51.02 51.02
    longitude  (y, x) float64 8MB 359.1 359.1 359.1 359.1 ... 5.865 5.873 5.88
Dimensions without coordinates: y, x
Data variables:
    q          (step, level, y, x) float64 141MB ...
    t          (step, level, y, x) float64 141MB ...
    u          (step, level, y, x) float64 141MB ...
    v          (step, level, y, x) float64 141MB ...
    wz         (step, level, y, x) float64 141MB ...
    z          (step, level, y, x) float64 141MB ...
Attributes:
    class:        d1
    stream:       oper
    levtype:      ml
    type:         fc
    expver:       aab0
    date:         20230820
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

In [103]:
height_levels=ds_point.z.data /g
target_height=30

date_time= pd.to_datetime("2023-08-20")

u_=list()
v_=list()
timestamp=list()

for s in range(0,3):
    time= date_time + pd.Timedelta(hours=s)
    
    ds_point = atm.isel(y=iy, x=ix, step=s)
    
    f = interp1d(height_levels, ds_point.u.values)
    u30= f(30)
    
    f = interp1d(height_levels, ds_point.v.values)
    v30= f(30)

    u_.append(u30)
    v_.append(v30)
    timestamp.append(time)
    
u_=np.array(u_)
v_=np.array(v_)

wind_speed_30 = np.sqrt(u_**2 + v_**2)
wind_dir = (np.degrees(np.arctan2(u_, v_)) + 180 ) % 360
df = pd.DataFrame({'wind_speed': wind_speed_30, 'wind_dir': wind_dir}, index=timestamp)
print(df)



                     wind_speed    wind_dir
2023-08-20 00:00:00    1.266321  262.486923
2023-08-20 01:00:00    0.843821  234.221910
2023-08-20 02:00:00    1.484247  208.428270


In [59]:
ds_point

<xarray.Dataset> Size: 1kB
Dimensions:    (level: 21)
Coordinates:
  * level      (level) int64 168B 70 71 72 73 74 75 76 ... 84 85 86 87 88 89 90
    latitude   float64 8B 51.05
    longitude  float64 8B 4.409
Data variables:
    q          (level) float64 168B 0.008526 0.008039 0.00756 ... 0.0116 0.01174
    t          (level) float64 168B 289.3 289.9 290.4 ... 292.0 292.1 292.2
    u          (level) float64 168B 6.3 6.47 6.383 6.092 ... 1.366 1.003 0.644
    v          (level) float64 168B 1.14 0.6114 0.1478 ... 0.1927 0.1034 0.03827
    wz         (level) float64 168B 0.01534 0.01102 ... 0.0003707 -4.363e-05
    z          (level) float64 168B 8.394e+03 7.693e+03 ... 188.0 70.52
Attributes:
    class:        d1
    stream:       oper
    levtype:      ml
    type:         fc
    expver:       aab0
    date:         20230820
    time:         0
    domain:       g
    Conventions:  CF-1.8
    institution:  ECMWF

np.float64(4.358788832279181)